In [1]:
import numpy as np
from ipywidgets import IntSlider

import plotlymath as plt
from myutils import interact, latex


In [2]:
plt.set_defaults(margin=(25,0,40,50))
colors = plt.plotly.colors.DEFAULT_PLOTLY_COLORS

In [3]:
def timeseries_interactive(A, initial_state, **options):
    max_steps = options.get("max_steps", 30)
    x, y = options.get("state_vars", ("x", "y"))
    xname, yname = options.get("var_names", ("$x$", "$y$"))
    solution = [initial_state]
    for n in range(max_steps):
        solution.append(A @ solution[-1])
    solution = np.insert(solution, 0, np.arange(max_steps + 1), axis=1)

    figure, plot = plt.make_figure(widget=True)
    figure.layout.update(width=750, height=400)
    figure.layout.xaxis.domain = (0, 0.85)
    plot.axes_labels(r"$n \text{ (time)}$", "Populations")
    plot.axes_ranges((0, max_steps + 1), (0, solution.max() * 1.05))

    @interact(n=IntSlider(min=0, max=max_steps, value=0, description="$n$"))
    def update(n):
        label = latex((f"{x}({n})", f"{y}({n})"))
        label += f" = {latex(solution[n,(1,2)], round=1)}"
        with figure.batch_update():
            plot.x_series = plt.points(solution[:n+1,(0,1)], name=xname, 
                    mode="markers+lines", color=colors[0], line_color="lightgray")
            plot.y_series = plt.points(solution[:n+1,(0,2)], name=yname, 
                    mode="markers+lines", color=colors[1], line_color="lightgray")
            plot.label = plt.text(f"${label}$", (0.87, 0.5), paper=True, font_size=24)

    return figure


In [4]:
def statespace_interactive(A, initial_states, **options):
    max_steps = options.get("max_steps", 30)
    xymax = options.get("xymax", None)
    x, y = options.get("state_vars", ("x", "y"))
    xname, yname = options.get("var_names", ("$x$", "$y$"))
    mode = "markers+lines" if options.get("connect", True) else "markers"
    solutions = {name: [state] for name, state in initial_states.items()}
    for solution in solutions.values():
        for t in range(max_steps):
            solution.append(A @ solution[-1])
    if xymax is None:
        xymax = np.array(list(solutions.values())).max() * 1.05

    figure, plot = plt.make_figure(widget=True)
    figure.layout.update(width=750, height=600)
    figure.layout.xaxis.domain = (0, 0.75)
    plot.axes_labels(fr"${x} \text{{ ({xname})}}$", fr"${y} \text{{ ({yname})}}$")
    plot.axes_ranges((0, xymax), (0, xymax), scale=(1, 1))
    evalues, T = np.linalg.eig(A)
    eigenlines = []
    for evalue, evector in zip(evalues, T.transpose()):
        eigenline = np.array([evector * -2*xymax, evector * 2*xymax])
        color = "darkred" if abs(evalue) > 1 else "darkgreen"
        eigenlines.append(plt.lines(eigenline, color=color, opacity=0.6, line_width=4, 
                name="Eigenline", legendgroup="eigenlines", visible="legendonly"))

    controls = {name: IntSlider(min=0, max=max_steps, value=0) for name in solutions}
    @interact(**controls)
    def update(**controls):
        with figure.batch_update():
            for i, (name, solution) in enumerate(solutions.items()):
                n = controls[name]
                label = latex((f"{x}({n})", f"{y}({n})"))
                label += f" = {latex(solution[n], round=1)}"
                setattr(plot, f"{name}points", plt.points(solution[:n+1], color=colors[i], 
                        mode=mode, line_color="lightgray", showlegend=False))
                setattr(plot, f"{name}label", plt.text(f"${label}$", (0.77, 0.85-0.2*i), 
                        color=colors[i], paper=True, size=24))

    update(**{name: 0 for name in solutions})
    if options.get("show_eigenlines", False):
        plot.eigenlines = eigenlines
    return figure


In [5]:
def complex_interactive(A):
    max_steps = 50
    solution = [np.array((10, 10), dtype=float)]
    for t in range(max_steps):
        solution.append(A @ solution[-1])

    figure, plot = plt.make_figure(widget=True)
    figure.layout.update(width=750, height=600)
    figure.layout.xaxis.domain = (0, 0.75)
    plot.axes_labels(r"$x$", r"$y$")
    plot.axes_ranges((-20, 20), (-20, 20), scale=(1, 1))
    options = dict(paper=True, font_size=24)
    evalues, evectors = np.linalg.eig(A)
    plot.matrix = plt.text(f"${latex(A, round=2)}$", (0.77, 0.95), **options)
    plot.label = plt.text(r"$\text{Eigenvalues:}$", (0.77, 0.8), **options)
    label = latex(evalues[0], round=3, conjpair=True)
    plot.eigen = plt.text(fr"$\lambda = {label}$", (0.77, 0.75), **options)
    label = latex(abs(evalues[0]), round=3)
    plot.abs = plt.text(fr"$\lvert \lambda \rvert = {label}$", (0.77, 0.7), **options)

    @interact(t=IntSlider(min=0, max=max_steps, value=0, description="$t$"))
    def update(t):
        with figure.batch_update():
            plot.solution = plt.points(solution[:t+1], color=colors[0], 
                    mode="markers+lines", line_color="lightgray")

    return figure


# A two-stage black bear population model

**Assumptions:** 
- The population is subdivided into two life stages: juveniles ($J$) and adults ($A$). 
- Each year, on average, $42\%$ of adults give birth to a cub. 
- Each year, $24\%$ of juveniles reach adulthood. 
- Each year, $15\%$ of adult bears die, and $29\%$ of juvenile bears die. 

**Resulting model:** 

<!-- $$ \begin{bmatrix} J(n+1) \\ A(n+1) \end{bmatrix} = \begin{bmatrix} 0.47 J(n) + 0.42 A(n) \\ 0.24 J(n) + 0.85 A(n) \end{bmatrix} $$ -->
$$ \begin{cases} J(n+1) = 0.47 J(n) + 0.42 A(n) \\ A(n+1) = 0.24 J(n) + 0.85 A(n) \end{cases} $$
or, in matrix form: 
$$ \begin{bmatrix} J(n+1) \\ A(n+1) \end{bmatrix} = \begin{pmatrix} 0.47 & 0.42 \\ 0.24 & 0.85 \end{pmatrix} \begin{bmatrix} J(n) \\ A(n) \end{bmatrix} $$


In [6]:
A = np.array(((0.47, 0.42), (0.24, 0.85)))
options = dict(state_vars=("J", "A"), var_names=("Juveniles", "Adults"))

timeseries_interactive(A, (500, 250), **options)

interactive(children=(IntSlider(value=0, description='$n$', max=30), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [7]:
statespace_interactive(A, {"$n$": (500, 250)}, connect=False, **options)

interactive(children=(IntSlider(value=0, description='$n$', max=30), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [8]:
initial_states = {
    "blue":   (500, 250), 
    "orange": (500,  50), 
    "green":  (150, 500), 
    "red":    ( 50, 700), 
}
statespace_interactive(A, initial_states, xymax=850, show_eigenlines=True, **options)

interactive(children=(IntSlider(value=0, description='blue', max=30), IntSlider(value=0, description='orange',…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [9]:
evalues, T = np.linalg.eig(A)
for evalue, evector in zip(evalues, T.transpose()):
    vector = np.round(-5*evector, 4)
    print(f"Eigenvalue λ = {evalue:.2f} with eigenvector {tuple(vector)}")


Eigenvalue λ = 0.29 with eigenvector (4.5957, -1.9696)
Eigenvalue λ = 1.03 with eigenvector (3.0, 4.0)


<br>
<br>
<br>
<br>
<br>

<br>
<br>
<br>
<br>
<br>

<br>
<br>
<br>
<br>
<br>



# A two-stage population model for cicadas

**Assumptions:** 
- The population is subdivided into two life stages: nymphs ($N$) and adult cicadas ($A$). 
- Nymphs remain underground for some years before emerging as adults. 

**Model, in matrix form:** 

<!-- $$ \begin{cases} N(t+1) = 0.488 N(t) + 1.632 A(t) \\ A(t+1) = 0.408 N(t) + 0.012 A(t) \end{cases} $$
or, in matrix form: -->
$$ \begin{bmatrix} N(t+1) \\ A(t+1) \end{bmatrix} = \begin{pmatrix} 0.488 & 1.632 \\ 0.408 & 0.012 \end{pmatrix} \begin{bmatrix} N(t) \\ A(t) \end{bmatrix} $$

**Questions:** 
- What percentage of nymphs remain underground as nymphs from one year to the next? 
- What percentage of adults survive a whole year, and thus remain as adults the following year? 
- What percentage of nymphs mature into adults (and thus emerge from underground) each year? 
- What is the birth rate? 
- What are the death rates of nymphs and adults? 


In [10]:
C = np.array([
    [0.488, 1.632],
    [0.408, 0.012], 
])

evalues, T = np.linalg.eig(C)
for evalue, evector in zip(evalues, T.transpose()):
    vector = np.round(np.sqrt(73)*evector, 4)
    print(f"Eigenvalue λ = {latex(evalue, round=2)} with eigenvector {tuple(vector)}")


Eigenvalue λ = 1.1 with eigenvector (8.0, 3.0)
Eigenvalue λ = -0.6 with eigenvector (-7.109, 4.7394)


In [11]:
options = dict(state_vars=("N", "A"), var_names=("Nymphs", "Adults"))
statespace_interactive(C, {"$n$": (0, 100)}, xymax=850, show_eigenlines=True, **options)

interactive(children=(IntSlider(value=0, description='$n$', max=30), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

# What about eigenvalues that are non-real? 

(Remember that, since the eigenvalues of a matrix are the roots of its characteristic polynomial, and a polynomial can have complex numbers as roots, even if a matrix has only real numbers in it, some of its eigenvalues can be non-real.) 


In [12]:
M = np.array([
    [ 0.6,  0.8], 
    [-0.8,  0.6], 
])
complex_interactive(M)

interactive(children=(IntSlider(value=0, description='$t$', max=50), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [13]:
M2 = np.array([
    [ 0.0, -1.0], 
    [ 1.0,  1.2], 
])
complex_interactive(M2)

interactive(children=(IntSlider(value=0, description='$t$', max=50), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [14]:
complex_interactive(0.9*M)

interactive(children=(IntSlider(value=0, description='$t$', max=50), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [15]:
complex_interactive(0.9*M2)

interactive(children=(IntSlider(value=0, description='$t$', max=50), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [16]:
complex_interactive(1.1*M)

interactive(children=(IntSlider(value=0, description='$t$', max=50), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…

In [17]:
complex_interactive(1.1*M2)

interactive(children=(IntSlider(value=0, description='$t$', max=50), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'line': {'color': 'lightgray'},
              'marker': {'color': 'rgb(31, 119, 1…